In this notebook, our goal is to transform the original dataset provided from LMS into the representation of the star schema which we designed.

### Libary and dataset imports

Here we import pandas and openpyxl, with the second one done by using pip since it gave us problems locally.

In [34]:
import pandas as pd
import numpy as np
import pip
pip.main(["install", "openpyxl"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple

Requirement already satisfied: openpyxl in /home/stas/.local/lib/python3.13/site-packages (3.1.5)

Requirement already satisfied: et-xmlfile in /home/stas/.local/lib/python3.13/site-packages (from openpyxl) (2.0.0)

0

Here we load the excel dataset.

In [35]:
file_path = "../../../00-Project/datasets/2024-08-01_LMS_data_2023.xlsx"
xls = pd.ExcelFile(file_path)

### Load the different sheets into their respective data frames

Here we load the different sheets in their own dataframes. We don't do it in one cell for testing reasons. \
Shipment is our fact, and we identified as dimensions the information about:
<ul>
  <li>Carrier</li>
  <li>Domain</li>
  <li>Country</li>
  <li>Service</li>
  <li>Customer</li>
  <li>Pickup address</li>
  <li>Delivery address</li>
</ul>

In [36]:
shipment_df = pd.read_excel(xls, 'shipment')

In [37]:
carrier_df = pd.read_excel(xls, 'carrier')
domain_df = pd.read_excel(xls, 'domain')
country_df = pd.read_excel(xls, 'country')

In [38]:
service_df = pd.read_excel(xls, 'service')

In [39]:
customer_df = pd.read_excel(xls, 'customer')

In [40]:
pickupaddress_df = pd.read_excel(xls, 'pickupaddress')
deliveryaddress_df = pd.read_excel(xls, 'deliveryaddress')

In [41]:
branchcode_df = pd.read_excel(xls, 'branchcode')
branchcode_customer_translation_df = pd.read_excel(xls, 'branchcode_customer_translation')

### Preprocessing of shipment

In the shipment dataframe, we make some quality checks for the shipment id

In [42]:
# We convert 'shipment_id' to numeric, then turn invalid values (non-numeric) to NaN
shipment_df['shipment_id'] = pd.to_numeric(shipment_df['shipment_id'], errors='coerce')

# We drop rows where 'shipment_id' is NaN
shipment_df_clean = shipment_df.dropna(subset=['shipment_id'])

### Creation of the shipment fact

In this section, we merge shipment with the domain in order to create the shipment fact.

In [43]:
# We merge the domain and the shipment, using 'domain_id' as key
fact_shipment = shipment_df_clean.merge(domain_df[['domain_id', 'name']], on='domain_id', how='left')

# We rename 'name' and 'bookingstate' for easier interpretation
fact_shipment = fact_shipment.rename(columns={'name': 'domain_name',
                                              'bookingstate': 'booking_state'})

We choose the relevant columns to keep, in accordance with the star schema model.

In [44]:
fact_shipment = fact_shipment[['shipment_id', 'customer_price', 'expected_carrier_price', 
                               'final_carrier_price', 'weight', 'shipment_type', 
                               'insurance_type', 'customer_id', 'pickupaddress_id', 
                               'deliveryaddress_id', 'service_id', 'domain_name', 
                               'pickup_date', 'delivery_date', 'real_pickup_date', 
                               'real_delivery_date', 'booking_state', 'lms_plus', 
                               'exworks_id','created_date']]

Here we calculate the margin of LMS on the shipments. In the documentation, they calculate the margin in the following way:
- customer_price - coalesce(final_carrier_price, expected_carrier_price)

Which means that it's the result of customer price minus the final carrier price. If the latter is not present, the expected carrier price will be used.

We then set margin to NaN in the following situations:
- when the shipment has been cancelled
- when the shipment is missing price data

In [45]:
# Here we convert all price columns to numeric, coercing errors to NaN
fact_shipment['customer_price'] = pd.to_numeric(fact_shipment['customer_price'], errors='coerce')
fact_shipment['final_carrier_price'] = pd.to_numeric(fact_shipment['final_carrier_price'], errors='coerce')
fact_shipment['expected_carrier_price'] = pd.to_numeric(fact_shipment['expected_carrier_price'], errors='coerce')

# Here we calculate carrier price using coalesce logic - use final_carrier_price if available, otherwise expected_carrier_price
carrier_price = fact_shipment['final_carrier_price'].fillna(fact_shipment['expected_carrier_price'])

# Here we calculate margin
fact_shipment['margin'] = fact_shipment['customer_price'] - carrier_price

# Here we set margin to NaN for cancelled shipments and for missing prices
fact_shipment.loc[fact_shipment['booking_state'] > 12, 'margin'] = np.nan  
fact_shipment.loc[fact_shipment['customer_price'].isna() | carrier_price.isna(), 'margin'] = np.nan 

### Process pickup and delivery address data frames

In this section we create the pickup and delivery address dimensions, considering the important columns and tying them to the shipment fact.

In [46]:
# We create the pickup dimension and we merge the domain name into it
dim_pickup_address = pickupaddress_df.merge(domain_df[['domain_id', 'name']], on='domain_id', how='left')
dim_pickup_address = dim_pickup_address.rename(columns={'name': 'domain_name'})

# We keep the columns present in the defined star schema
dim_pickup_address = dim_pickup_address[['pickupaddress_id', 'created_date', 'domain_name', 'country_id', 'postal_code', 'city']]
dim_pickup_address = dim_pickup_address.rename(columns={'pickupaddress_id': 'pickup_address_id'})

# We create the pickup dimension and we merge the domain name into it
dim_delivery_address = deliveryaddress_df.merge(domain_df[['domain_id', 'name']], on='domain_id', how='left')
dim_delivery_address = dim_delivery_address.rename(columns={'name': 'domain_name'})

# We keep the columns present in the defined star schema
dim_delivery_address = dim_delivery_address[['deliveryaddress_id', 'created_date', 'domain_name', 'country_id', 'postal_code', 'city']]
dim_delivery_address = dim_delivery_address.rename(columns={'deliveryaddress_id': 'delivery_address_id'})

# We rename the columns in the shipment fact table for better understanding
fact_shipment = fact_shipment.rename(columns={'pickupaddress_id': 'pickup_address_id', 
                                                'deliveryaddress_id': 'delivery_address_id'})

# Convert datetime while preserving the time component
dim_delivery_address['created_date'] = pd.to_datetime(dim_delivery_address['created_date'], errors='coerce')
dim_pickup_address['created_date'] = pd.to_datetime(dim_pickup_address['created_date'], errors='coerce')

### Process customer data frame

In this section we create the customer dimensions; we handle it's relationships with the branchcodes, other than the master relationships.

Customers are organized in a hierarchical structure where:
<ul>
  <li>Master Accounts are identified when a customer's sequence number matches their structure number</li>
  <li>Industry Classifications are assigned through branchcodes:</li>
  <ul>
    <li>Each customer can have multiple branchcodes</li>
    <li>Only the first/primary branchcode is used as the main industry, meaning the main activity that the customer does. We will consider it in order to have a 1 to n relationship</li>
    <li>Each branchcode has both a specific industry name and a broader sector classification</li>
</ul>
</ul>

First we handle the customer and the industries; we merge the branchcode and the root branch (which we refer to using the sector). \
We get the main industry for each customer by using the first branchcode.

In [47]:
# We join the customer with the branchcode table to get both the specific industry name and its root info
customer_industries = (
    customer_df.merge(
        # First we get the translation table to link customers to branchcodes
        branchcode_customer_translation_df,
        on='customer_id',
        how='left'
    )
    # Then we get the branchcode information, including the branch name
    .merge(
        branchcode_df[['branchcode_id', 'branch_name', 'root_branch_id']],
        on='branchcode_id',
        how='left'
    )
    # At last, we get the root branch name by joining branchcode table again
    .merge(
        branchcode_df[['branchcode_id', 'branch_name']],
        left_on='root_branch_id',
        right_on='branchcode_id',
        how='left',
        suffixes=('', '_root')
    )
    # We sort by translation ID to ensure that the main industry comes first
    .sort_values('branchcode_customer_id')
)

Next, we get only the main industry name and its root branch name for each customer

In [48]:
customer_industry_info = customer_industries.groupby('customer_id').agg({
    'branch_name': 'first',          # We get the main industry name
    'branch_name_root': 'first'      # We get the root branch name
}).reset_index()

# Now we can create the customer dimension with all the information 
dim_customer = (
    customer_df
    # Here we merge the industry information
    .merge(
        customer_industry_info[['customer_id', 'branch_name', 'branch_name_root']], 
        on='customer_id',
        how='left'
    )
    # Here we merge the domain information
    .merge(
        domain_df[['domain_id', 'name']],
        on='domain_id',
        how='left'
    )
    # Lastly we rename some of the columns in order to be more descriptive
    .rename(columns={
        'name': 'domain_name',
        'sequencenumber': 'sequence_number',
        'structurenumber': 'structure_number',
        'branch_name': 'main_industry_name',
        'branch_name_root': 'industry_sector_name'
    })
)

As the last step, we add for each customer a column displaying if it is the 'master', meaning the main entity of the sequence and structure number hierarchy.

In [49]:
dim_customer['is_master'] = dim_customer['sequence_number'] == dim_customer['structure_number']
dim_customer = dim_customer[[
    'customer_id', 'created_date', 'domain_name', 
    'main_industry_name', 'industry_sector_name',
    'segmentation', 'sequence_number', 'structure_number', 
    'is_master'
]]

# Convert to datetime while preserving time if it exists
dim_customer['created_date'] = pd.to_datetime(dim_customer['created_date'], errors='coerce')

### Process dates data frames

In this section we create the date dimension define through primary keys how it's tied to the shipment fact. \
Since in a previous iteration we were dropping rows where one of the date columns was NaT, and it was resulting in information loss for some categories of service type, we decided to take a different approach.

We are going to use a sentinel date which will have NaT in the date dimension, and assign this date to all the values in the shipment where a date is not present, instead of outright dropping them. This way we can preserve information while handling the problematic values.



In this step we define the date columns present in shipment and we initialize the date dimension with the sentinel date, to which we give ID 0.

In [50]:
# We group the relevant date columns
date_columns = ['created_date', 'pickup_date', 'real_pickup_date', 'delivery_date', 'real_delivery_date']

# We initialize the date dimension with the missing date entry
date_dim = pd.DataFrame({
    'full_date': [pd.NaT], # we use NaT for the missing date
    'year': [np.nan],
    'month': [np.nan],
    'quarter': [np.nan],
    'is_missing': [True]
})

date_dim['date_id'] = 0 

Next, we process the values in each column, find the valid ones, create a dataframe with the other derived features and then append it to the date dimension. 

In [51]:
# We process each date column
for col in date_columns:
    # We convert to datetime, keeping invalid as NaT
    fact_shipment[col] = pd.to_datetime(fact_shipment[col], errors='coerce')
    
    # W get unique valid dates
    valid_dates = fact_shipment[col].dropna().unique()
    
    if len(valid_dates) > 0:
        new_dates = pd.DataFrame({
            'full_date': valid_dates,  # Keep the full datetime
            'year': [d.year for d in valid_dates],
            'month': [d.month for d in valid_dates],
            'quarter': [(d.month-1)//3 + 1 for d in valid_dates],
            'is_missing': False
        })
        
        # We append the result to date dimension dropping duplicates
        date_dim = pd.concat([date_dim, new_dates]).drop_duplicates(subset=['full_date']).reset_index(drop=True)

In this section we assign IDs to the date dimension, reserving value 0 for the sentinel date, after which we update the fact table with the proper IDs of each date column. At last, we drop the respective columns.

In [52]:
# Assign proper IDs
date_dim.loc[date_dim['is_missing'] == False, 'date_id'] = range(1, len(date_dim[date_dim['is_missing'] == False]) + 1)

# Update fact table with date IDs
for col in date_columns:
    # Convert dates to datetime maintaining the time component
    fact_shipment[col] = pd.to_datetime(fact_shipment[col], errors='coerce')
    
    # Create a mapping series that's null for missing dates
    fact_shipment[f'{col}_date'] = fact_shipment[col]
    
    # Merge with date dimension to get IDs
    fact_shipment = fact_shipment.merge(
        date_dim[['full_date', 'date_id']],
        left_on=f'{col}_date',
        right_on='full_date',
        how='left'
    )
    
    # Fill missing date_ids with 0 (our missing date ID)
    fact_shipment[f'{col}_id'] = fact_shipment['date_id'].fillna(0)
    
    # Drop the not needed columns
    fact_shipment = fact_shipment.drop(columns=[col, f'{col}_date', 'full_date', 'date_id'])

# Drop is_missing column
date_dim = date_dim.drop(columns=['is_missing'])

### Process service dataframe

In this section we create the service dimension; we merge the domain information into the service and handle the naming alongside the importanct columns.

In [53]:
# Merge domain name into the service dimension table, adding suffixes to distinguish between columns
dim_service = service_df.merge(domain_df[['domain_id', 'name']], on='domain_id', how='left', suffixes=('_service', '_domain'))

# Rename the different columns for clarity and formatting
dim_service = dim_service.rename(columns={
    'name_domain': 'domain_name',
    'name_service': 'name',
    'servicetype': 'service_type',
    'transporttype': 'transport_type'
})

# Keep only the relevant columns
dim_service = dim_service[['service_id', 'created_date', 'name', 'service_type', 'transport_type', 'carrier_id', 'domain_name']]

# Convert to datetime preserving the time component
dim_service['created_date'] = pd.to_datetime(dim_service['created_date'], errors='coerce')

### Process carrier and country dimensions

In this section we create the carrier and country dimensions, merging the carrier with the domain information and keeping the relevant columns.

In [54]:
# Merge the domain name into the service dimension table, adding suffixes to distinguish between columns
dim_carrier = carrier_df.merge(domain_df[['domain_id', 'name']], on='domain_id', how='left', suffixes=('_carrier', '_domain'))

# Rename some of the columns for clarity and formatting reasons
dim_carrier = dim_carrier.rename(columns={
    'name_carrier': 'name',
    'name_domain': 'domain_name'
})

# Filter the important columns
dim_carrier = dim_carrier[['carrier_id', 'name', 'created_date', 'domain_name']]

# Convert to datetime preserving any time components
dim_carrier['created_date'] = pd.to_datetime(dim_carrier['created_date'], errors='coerce')

In [55]:
# Here we create the country dimension, filtering the needed columns
dim_country = country_df[['country_id', 'name', 'isocountrycode', 'continent', 'EU']]

# Here we rename the 'isocountrycode' column for clarity
dim_country = dim_country.rename(columns={'isocountrycode': 'iso_country_code'})

### Save new start schema dataset

The last operation left to do is to merge toeghter the fact table and different dimensions into a single excel file.

In [56]:
with pd.ExcelWriter('../../../00-Project/datasets/star_schema_dataset_1.xlsx', engine='xlsxwriter') as writer:
    # The Fact table
    fact_shipment.to_excel(writer, sheet_name='fact_shipment', index=False)
    
    # The Dimension tables
    dim_customer.to_excel(writer, sheet_name='dim_customer', index=False)
    dim_delivery_address.to_excel(writer, sheet_name='dim_delivery_address', index=False)
    dim_pickup_address.to_excel(writer, sheet_name='dim_pickup_address', index=False)
    date_dim.to_excel(writer, sheet_name='dim_date', index=False)
    dim_service.to_excel(writer, sheet_name='dim_service', index=False)
    dim_carrier.to_excel(writer, sheet_name='dim_carrier', index=False)
    dim_country.to_excel(writer, sheet_name='dim_country', index=False)

print("Star schema transformation with domain names included completed!")

Star schema transformation with domain names included completed!
